In [13]:
# from autogluon.tabular import TabularPredictor
# import pandas as pd
# import numpy as np
# import matplotlib.pyplot as plt
# import seaborn as sns
# from autogluon.tabular import TabularPredictor
# from sklearn.model_selection import KFold
# from sklearn.metrics import roc_auc_score
# from sklearn.preprocessing import label_binarize
# from autogluon.core.metrics import make_scorer

# def custom_roc_auc_ovr(y_true, y_pred_proba, **kwargs):
#     binary_labels = label_binarize(y_true, classes=np.unique(y_true))
#     ovr_auc = roc_auc_score(binary_labels, y_pred_proba, average='macro', multi_class='ovr')
#     return ovr_auc

# # Convert custom metrics into an AutoGluon usable rater
# custom_auc_scorer = make_scorer(name='roc_auc_ovr', score_func=custom_roc_auc_ovr, greater_is_better=True, needs_proba=True)
# from autogluon.tabular import TabularPredictor
# import pandas as pd

# # 初始化一个空的 DataFrame 来存储每个折次的 leaderboard
# leaderboards = []

# # 循环加载每个折次的模型并获取其 leaderboard
# for fold in range(10):
#     model_path = f'/home/tailab/se/solarelastosis_v1/code/solar_elastosis_data_processor/autogluon_three_ways_fold_{fold}_model'
#     predictor = TabularPredictor.load(model_path)
#     leaderboard = predictor.leaderboard(silent=True)
#     leaderboard['fold'] = fold  # 添加折次信息，方便后续分析
#     leaderboards.append(leaderboard)

# # 将所有折次的 leaderboard 合并为一个 DataFrame
# combined_leaderboard = pd.concat(leaderboards, ignore_index=True)

# # 过滤掉非数值列，仅对数值列计算平均得分
# numeric_columns = combined_leaderboard.select_dtypes(include=['number']).columns
# average_leaderboard = combined_leaderboard.groupby('model')[numeric_columns].mean().reset_index()

# # 按照验证集分数进行排序
# average_leaderboard = average_leaderboard.sort_values(by='score_val', ascending=False)

# # 输出平均 leaderboard
# print(average_leaderboard)



In [14]:
import torch
import pandas as pd
import os
import regex as re
import numpy as np
import shutil

In [15]:
gem_csv_path = "/home/tailab/se/data/Gem_processed_new.csv"
pt_path = "/home/tailab/se/data/resnet_features_solarelastosis/tumor_subtyping_resnet_features/pt_files/"
h5_path = "/home/tailab/se/data/resnet_features_solarelastosis/tumor_subtyping_resnet_features/h5_files/"

In [ ]:
df = pd.read_csv(gem_csv_path)

# Specify the indices of continuous columns from the original DataFrame (6th, 7th, and 34th columns)
continuous_indices = [5, 6, 33]  # Adjusted for zero-indexing

# Extract the names of continuous columns based on original indices before dropping any columns
continuous_columns = [
    df.columns[idx] for idx in continuous_indices if idx < df.shape[1]
]
df.drop_duplicates(subset =['Deepest Lesion (Recent & Previous) blgm'],keep='first',inplace=True)
file_ref = df['Deepest Lesion (Recent & Previous) blgm'].values.tolist()
target_ref = dict(zip(df['Deepest Lesion (Recent & Previous) blgm'].values.tolist(), df['Deepest Lesion (Recent & Previous) SOLAREL'].values.tolist()))
# List of columns to remove after identifying continuous columns
columns_to_remove = [
    'MRN', 'status', 'Indicator for being a control to case patient', 
    'Deepest Lesion (Recent & Previous) SOLAREL', 
    'Deepest Lesion (Recent & Previous) LYMPHCT_new',
    'blgm_1', 'blgm_2', 'Deepest Lesion (Recent & Previous) LYMPHCT', 
    'Deepest Lesion (Recent & Previous) Solar_new',
    'Solar Elastosis - Control/Case Defining Lesion - Most Recent Dx', 
    'Solar Elastosis Lesion 1 recoded with absent=0', 
    'ulc1', 'ulc2', 'Pathologist performing Slide Review - Control/Case Defining Lesion - Most Recent Dx',
    'Deepest Lesion (Recent & Previous) reviewed?', 
    'Dx most recent - calculated from DOB and path report'
]

# Drop columns that are present in the DataFrame
present_columns_to_remove = [col for col in columns_to_remove if col in df.columns]
refined_df = df.drop(present_columns_to_remove, axis=1)
first_column = refined_df.pop('Deepest Lesion (Recent & Previous) blgm')
# leave the columns still exists after dropping
continuous_columns = [col for col in refined_df.columns if col in continuous_columns]
# Identify categorical columns as those not in the continuous columns list
categorical_columns = [col for col in refined_df.columns if col not in continuous_columns]

# Final identified columns for categorical and continuous data
final_categorical_columns = categorical_columns
final_continuous_columns = continuous_columns

# Display or use the identified categorical and continuous columns as needed
print("Categorical Columns:", final_categorical_columns)
print("Continuous Columns:", final_continuous_columns)


refined_df.to_csv('refined_gem.csv')
#columns, index = refined_df.columns, refined_df.index
#from sklearn.preprocessing import StandardScaler
#refined_df = pd.DataFrame(StandardScaler().fit_transform(refined_df), columns=columns, index=index)
data = refined_df
all_columns = data.columns
# # Drop the unnamed index column if present
# if 'Unnamed: 0' in data.columns:
#     data = data.drop(columns=['Unnamed: 0'])

# # Threshold for determining initial categorical columns
# categorical_threshold = 100  # Columns with unique values <= 100 will be considered initially as categorical

# # Identify initial categorical and continuous columns
# categorical_columns = data.columns[data.nunique() <= categorical_threshold]
# continuous_columns = data.columns.difference(categorical_columns)

# # Further refine categorical columns by ensuring they are integer-like
# final_categorical_columns = []
# final_continuous_columns = list(continuous_columns)  # Start with initially identified continuous columns

# for col in categorical_columns:
#     # Check if all values are integers or floats that are effectively integers (e.g., 1.0, 2.0)
#     if np.issubdtype(data[col].dtype, np.number) and all(data[col] % 1 == 0):
#         final_categorical_columns.append(col)
#     else:
#         final_continuous_columns.append(col)

# # Convert categorical columns to integers if they are not already
# for col in final_categorical_columns:
#     data[col] = data[col].astype(int)
#     data[col] = pd.Categorical(data[col]).codes

# Splitting the data into final categorical and continuous sets
categorical_data = data[final_categorical_columns]
continuous_data = data[final_continuous_columns]

# Normalizing continuous data
mean = continuous_data.mean(axis=0)
std = continuous_data.std(axis=0)
continuous_data_normalized = (continuous_data - mean) / (std + 1e-9)
# Combine categorical and normalized continuous data
processed_data = pd.concat([categorical_data, continuous_data_normalized], axis=1)

# Final processed data
refined_df = processed_data

#print(refined_df)


print("Final input data shape:", processed_data.shape)
# for col in final_categorical_columns:
#     unique_count = data[col].nunique()
#     print(f'{unique_count},')




In [ ]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder, StandardScaler
# from pytorch_tabnet.tab_model import TabNetClassifier
import torch
import numpy as np
from sklearn.model_selection import KFold
# from pytorch_tabnet.augmentations import ClassificationSMOTE
import scipy
from sklearn.metrics import roc_auc_score

# Step 1: Label Encoding for Categorical Features
categorical_cols = refined_df.columns[:-3]  # Assuming last 3 columns are numerical
# refined_df_categorical = refined_df[categorical_cols]
refined_df_categorical = pd.get_dummies(refined_df[categorical_cols], columns=categorical_cols, dtype=int)
refined_df_numerical = refined_df.iloc[:, -3:]  # Keeping numerical data

# Combine encoded categorical data and normalized numerical data
refined_df = pd.concat([refined_df_categorical, refined_df_numerical], axis=1)

# Convert DataFrame to numpy array for TabNet
data_np = refined_df.values

data_pd = pd.DataFrame(data_np)
data_pd.insert(0, 'Deepest Lesion (Recent & Previous) blgm', first_column)
data_pd.dropna(inplace=True)

# Prepare target labels based on refined_target_map
labels = []
filtered_indices = []

# and  solar elastosis levels




for idx, id in enumerate(data_pd['Deepest Lesion (Recent & Previous) blgm']):
    if not pd.isna(id):
        label = target_ref.get(id, None)  # Use .get() to avoid KeyErrors
        if label == 3:
            labels.append(2)  # 3 = Absent"
            filtered_indices.append(idx)
        if label == 2:
            labels.append(1)  # 2 = Marked
            filtered_indices.append(idx)
        elif label == 1:
            labels.append(0)  # "1 = Mild/moderate
            filtered_indices.append(idx)
        else:
            continue

# Filter data_np to keep only rows corresponding to selected labels
data_np = data_np[filtered_indices]
labels = np.array(labels)

print(data_np.shape, labels.shape)


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from autogluon.tabular import TabularPredictor
from sklearn.model_selection import KFold
from sklearn.metrics import roc_auc_score, average_precision_score
from autogluon.core.metrics import make_scorer

# Convert data to DataFrame for AutoGluon
train_df = pd.DataFrame(data_np)
train_df['target'] = labels

# Custom ROC AUC for binary classification
def custom_roc_auc_ovr(y_true, y_pred_proba, **kwargs):
    # ovr_auc = roc_auc_score(y_true, y_pred_proba[:, 1])  # Use the probability of the positive class
    ovr_auc = roc_auc_score(y_true, y_pred_proba, multi_class='ovr')  # Use the probability of the positive class
    return ovr_auc

# Wrap the custom AUC function into an AutoGluon scorer
custom_auc_scorer = make_scorer(
    name='custom_roc_auc_ovr',
    score_func=custom_roc_auc_ovr,
    greater_is_better=True,
    needs_proba=True
)

# Initialize KFold for cross-validation (10 folds)
kf = KFold(n_splits=10, shuffle=True, random_state=42)
embeddings_dict = {}
test_results = []  # List to store the results of each fold (model, AUC, AUC-PR)

# Perform 10-fold cross-validation: 8 folds for training, 1 fold for validation, and 1 fold for testing
for fold, (train_val_index, test_index) in enumerate(kf.split(train_df)):
    # Split into train+val and test sets
    train_val_data = train_df.iloc[train_val_index]
    test_data = train_df.iloc[test_index]

    test_features = test_data.drop(columns=['target'])
    test_labels = test_data['target'].values  # Ensure test_labels is a 1D numpy array

    # Further split train_val_data into 8 folds for training and 1 fold for validation
    inner_kf = KFold(n_splits=9, shuffle=True, random_state=fold)
    train_index, val_index = next(inner_kf.split(train_val_data))

    fold_train_data = train_val_data.iloc[train_index]
    val_data = train_val_data.iloc[val_index]
    
    val_features = val_data.drop(columns=['target'])
    val_labels = val_data['target'].values  # Ensure val_labels is a 1D numpy array

     # Define model path for the current fold
    model_path = f'/home/tailab/se/data/autoGulon_results/solar_elastosis_data_processor/threeway_fold_{fold}_model'

    # Initialize and fit AutoGluon Tabular Predictor if no existing model is found
    predictor = TabularPredictor(
        label='target', 
        eval_metric=custom_auc_scorer,
        path=model_path
    )
    predictor.fit(train_data=fold_train_data, time_limit=3600, presets='best_quality')

    # Step 3: Evaluate all models on the validation set to select the best one based on AUC
    model_names = predictor.get_model_names()  # Get all trained models
    best_model_name = None
    best_val_auc = -np.inf
    for model in model_names:
        val_pred_proba = predictor.predict_proba(val_features, model=model, as_pandas=False)
        # auc_score_val = roc_auc_score(val_labels, val_pred_proba[:, 1])  # AUC for the positive class
        auc_score_val = roc_auc_score(val_labels, val_pred_proba, multi_class='ovr')  # AUC for the positive class
        
        if auc_score_val > best_val_auc:
            best_val_auc = auc_score_val
            best_model_name = model

    # Step 4: Use the best model (on validation) to predict on the test set
    test_pred_proba = predictor.predict_proba(test_features, model=best_model_name, as_pandas=False)

    # Calculate AUC and AUC-PR for the positive class on the test set
    auc_score_test = roc_auc_score(test_labels, test_pred_proba[:, 1])  # AUC for the positive class
    # auc_score_test = roc_auc_score(test_labels, test_pred_proba, multi_class='ovr')  # AUC for the positive class
    aucpr_score_test = average_precision_score(test_labels, test_pred_proba[:, 1])  # AUC-PR for the positive class
    # aucpr_score_test = average_precision_score(test_labels, test_pred_proba, average='macro')  # AUC-PR for the positive class

    # Save the AUC and AUC-PR scores for the current fold along with the best model name
    test_results.append({
        'fold': fold,
        'model_name': best_model_name,
        'val_auc': best_val_auc,
        'test_auc': auc_score_test,
        'test_aucpr': aucpr_score_test
    })

    # Step 5: Extract features (embeddings) from the test set
    for idx, embedding in zip(test_features.index, test_pred_proba):
        embeddings_dict[idx] = embedding

# Get the sorted indices of test_features
sorted_indices = sorted(embeddings_dict.keys())

# Reassemble embeddings in the sorted order
final_embeddings = np.array([embeddings_dict[idx] for idx in sorted_indices])

print(f"Final embeddings shape: {final_embeddings.shape}")

# Convert test results to DataFrame and save to CSV
test_results_df = pd.DataFrame(test_results)
print(f"Test Results for each fold:\n{test_results_df}")
test_results_df.to_csv('test_results_with_models.csv', index=False)

# Display final results
leaderboard = predictor.leaderboard(silent=False)
print("\nLeaderboard of Models:")
print(leaderboard)

# Save leaderboard to CSV
leaderboard.to_csv('autogluon_leaderboard.csv', index=False)

# Visualize the results
plt.figure(figsize=(12, 6))
sns.barplot(x='model', y='score_val', data=leaderboard, ci=None)
plt.title('Validation Score by Model')
plt.ylabel('Validation Score (AUC)')
plt.xlabel('Model')
plt.xticks(rotation=45)
plt.show()

plt.figure(figsize=(12, 6))
sns.barplot(x='model', y='fit_time', data=leaderboard, ci=None)
plt.title('Training Time by Model')
plt.ylabel('Training Time (seconds)')
plt.xlabel('Model')
plt.xticks(rotation=45)
plt.show()

plt.figure(figsize=(12, 6))
sns.barplot(x='model', y='pred_time_val', data=leaderboard, ci=None)
plt.title('Prediction Time by Model')
plt.ylabel('Prediction Time (seconds)')
plt.xlabel('Model')
plt.xticks(rotation=45)
plt.show()


In [1]:
# embedding part


# import pandas as pd
# import numpy as np
# import matplotlib.pyplot as plt
# import seaborn as sns
# from autogluon.tabular import TabularPredictor
# from sklearn.model_selection import KFold
# from sklearn.metrics import roc_auc_score, average_precision_score
# from autogluon.core.metrics import make_scorer

# # Convert data to DataFrame for AutoGluon
# train_df = pd.DataFrame(data_np)
# train_df['target'] = labels

# # Custom ROC AUC for binary classification
# def custom_roc_auc_ovr(y_true, y_pred_proba, **kwargs):
#     ovr_auc = roc_auc_score(y_true, y_pred_proba[:, 1])  # Use the probability of the positive class
#     return ovr_auc

# # Wrap the custom AUC function into an AutoGluon scorer
# custom_auc_scorer = make_scorer(
#     name='custom_roc_auc_ovr',
#     score_func=custom_roc_auc_ovr,
#     greater_is_better=True,
#     needs_proba=True
# )

# # Initialize KFold for cross-validation (10 folds)
# kf = KFold(n_splits=10, shuffle=True, random_state=42)
# embeddings_dict = {}
# test_results = []  # List to store the results of each fold (model, AUC, AUC-PR)

# # Perform 10-fold cross-validation: 8 folds for training, 1 fold for validation, and 1 fold for testing
# for fold, (train_val_index, test_index) in enumerate(kf.split(train_df)):
#     # Split into train+val and test sets
#     train_val_data = train_df.iloc[train_val_index]
#     test_data = train_df.iloc[test_index]

#     test_features = test_data.drop(columns=['target'])
#     test_labels = test_data['target'].values  # Ensure test_labels is a 1D numpy array

#     # Further split train_val_data into 8 folds for training and 1 fold for validation
#     inner_kf = KFold(n_splits=9, shuffle=True, random_state=fold)
#     train_index, val_index = next(inner_kf.split(train_val_data))

#     fold_train_data = train_val_data.iloc[train_index]
#     val_data = train_val_data.iloc[val_index]
    
#     val_features = val_data.drop(columns=['target'])
#     val_labels = val_data['target'].values  # Ensure val_labels is a 1D numpy array

#     # Step 2: Initialize AutoGluon Tabular Predictor and fit on training data
#     predictor = TabularPredictor(
#         label='target', 
#         eval_metric='roc_auc',   # Use the correct scorer
#         path=f'marked_vs_mild_fold_{fold}_model'
#     )

#     # Fit the model on the training data (8/9)
#     predictor.fit(train_data=fold_train_data, time_limit=3600, presets='best_quality')

#     # Step 3: Evaluate all models on the validation set to select the best one based on AUC
#     model_names = predictor.get_model_names()  # Get all trained models
#     best_model_name = None
#     best_val_auc = -np.inf
#     for model in model_names:
#         val_pred_proba = predictor.predict_proba(val_features, model=model, as_pandas=False)
#         auc_score_val = roc_auc_score(val_labels, val_pred_proba[:, 1])  # AUC for the positive class
        
#         if auc_score_val > best_val_auc:
#             best_val_auc = auc_score_val
#             best_model_name = model

#     # Step 4: Use the best model (on validation) to predict on the test set
#     test_pred_proba = predictor.predict_proba(test_features, model=best_model_name, as_pandas=False)

#     # Calculate AUC and AUC-PR for the positive class on the test set
#     auc_score_test = roc_auc_score(test_labels, test_pred_proba[:, 1])  # AUC for the positive class
#     aucpr_score_test = average_precision_score(test_labels, test_pred_proba[:, 1])  # AUC-PR for the positive class

#     # Save the AUC and AUC-PR scores for the current fold along with the best model name
#     test_results.append({
#         'fold': fold,
#         'model_name': best_model_name,
#         'val_auc': best_val_auc,
#         'test_auc': auc_score_test,
#         'test_aucpr': aucpr_score_test
#     })

#     # Step 5: Extract features (embeddings) from the test set
#     for idx, embedding in zip(test_features.index, test_pred_proba):
#         embeddings_dict[idx] = embedding

# # Get the sorted indices of test_features
# sorted_indices = sorted(embeddings_dict.keys())

# # Reassemble embeddings in the sorted order
# final_embeddings = np.array([embeddings_dict[idx] for idx in sorted_indices])

# print(f"Final embeddings shape: {final_embeddings.shape}")

# # Convert test results to DataFrame and save to CSV
# test_results_df = pd.DataFrame(test_results)
# print(f"Test Results for each fold:\n{test_results_df}")
# test_results_df.to_csv('test_results_with_models.csv', index=False)

# # Display final results
# leaderboard = predictor.leaderboard(silent=False)
# print("\nLeaderboard of Models:")
# print(leaderboard)

# # Save leaderboard to CSV
# leaderboard.to_csv('autogluon_leaderboard.csv', index=False)

# # Visualize the results
# plt.figure(figsize=(12, 6))
# sns.barplot(x='model', y='score_val', data=leaderboard, ci=None)
# plt.title('Validation Score by Model')
# plt.ylabel('Validation Score (AUC)')
# plt.xlabel('Model')
# plt.xticks(rotation=45)
# plt.show()

# plt.figure(figsize=(12, 6))
# sns.barplot(x='model', y='fit_time', data=leaderboard, ci=None)
# plt.title('Training Time by Model')
# plt.ylabel('Training Time (seconds)')
# plt.xlabel('Model')
# plt.xticks(rotation=45)
# plt.show()

# plt.figure(figsize=(12, 6))
# sns.barplot(x='model', y='pred_time_val', data=leaderboard, ci=None)
# plt.title('Prediction Time by Model')
# plt.ylabel('Prediction Time (seconds)')
# plt.xlabel('Model')
# plt.xticks(rotation=45)
# plt.show()


In [ ]:
print(final_embeddings.shape)

refined_df = pd.DataFrame(data=final_embeddings)

refined_df.insert(0,'Deepest Lesion (Recent & Previous) blgm',first_column)
print(refined_df)

In [ ]:
print(len(df.columns.to_list()))
print(len(refined_df.columns.to_list()))
print(refined_df.columns.to_list())

In [9]:
pt_files = os.listdir(pt_path)
h5_files = os.listdir(h5_path)
file_map = {}
h5_map ={}
for file in pt_files:
    file_number = int(re.findall(r'\d+',file)[0])
    file_map [file_number] = file 
    h5_map [file_number] = file.replace('.pt','.h5')

In [ ]:
print(refined_df)

In [ ]:
concatenate = False
no_match_refs = []
refined_ptfile_map = {}
refined_h5file_map ={}
refined_target_map ={}

tab_data = {}
refined_file_ref =[]

for ref in file_ref:
    if ref in file_map.keys():
        refined_h5file_map [ref] = h5_map[ref]
        refined_ptfile_map [ref] = file_map[ref]
        refined_target_map [ref] = target_ref[ref]
        filtered_rows = refined_df[refined_df['Deepest Lesion (Recent & Previous) blgm'] == ref]
        if not filtered_rows.empty:
            #print(filtered_rows.values.tolist()[0][1:]) 
            tab_data[ref] = filtered_rows.values.tolist()[0][1:]
            refined_file_ref.append(ref)
        else:
            print(f"No match found for ref: {ref}")
            no_match_refs.append(ref)


In [ ]:
print(len(refined_ptfile_map))
print(len(refined_target_map))
print(len(tab_data))
#refined_df.to_csv('/home/tailab/se/data/tumor_vs_normal_dummy_clean_tabular.csv')

In [13]:
new_pt_path = "/home/tailab/se/data/autoML_mild_vs_absent_features/pt_files/"
new_h5_path = "/home/tailab/se/data/autoML_mild_vs_absent_features/h5_files/"

if not os.path.exists(new_pt_path):
    os.makedirs(new_pt_path)
if not os.path.exists(new_h5_path):
    os.makedirs(new_h5_path)

In [ ]:
def append_embeddings(embeddings, data):
    embeddings = embeddings.cpu().detach().numpy()
    data = np.array([data])
    return torch.from_numpy(np.hstack((embeddings,np.broadcast_to(data,(embeddings.shape[0],data.shape[1])))))

from tqdm import tqdm
concatenate = True
for ref in tqdm(refined_file_ref):
    pt_embeddings = torch.load(os.path.join(pt_path,refined_ptfile_map[ref]))
    temp_tab_data = tab_data[ref]
    #print(len(temp_tab_data))
    #continue
    if concatenate:
        concat_embeddings = append_embeddings(pt_embeddings,temp_tab_data)
        concat_embeddings = concat_embeddings.float()
    else:
        concat_embeddings = torch.from_numpy(np.array([temp_tab_data]))
        concat_embeddings = concat_embeddings.float()
        #print(concat_embeddings.shape)
        
    torch.save(concat_embeddings,os.path.join(new_pt_path,refined_ptfile_map[ref]))
    shutil.copy(os.path.join(h5_path,refined_h5file_map[ref]),os.path.join(new_h5_path, refined_h5file_map[ref]))

 

    

In [20]:
new_target_df = pd.DataFrame(columns=['case_id','slide_id','label'])
# {'subtype_1':0, 'subtype_2':1, 'subtype_3':2}
# for ref in refined_file_ref:
#     label = refined_target_map[ref]
#     if label == 1:
#         label = 'mild'
#     elif label == 3:
#         label = 'absent'
#     else:
#         #label = "subtype_1"
#         continue
for ref in refined_file_ref:
    label = refined_target_map[ref]
    if label == 3:
        label = 'subtype_3'
    elif label == 1:
        label = 'subtype_1'
    elif label == 2:
        label = 'subtype_2'
    else:
        #label = "subtype_1"
        continue
    new_target_df=new_target_df._append({'case_id': ref, 'slide_id':refined_h5file_map[ref].replace('.h5',''),'label':label},ignore_index =True)


In [ ]:
print(new_target_df)

In [17]:
new_target_df = new_target_df[~new_target_df['case_id'].isin(no_match_refs)]
new_target_df.to_csv("/home/tailab/se/data/tumor_vs_normal_mild_vs_absent.csv",index=False)

In [ ]:
print(new_target_df['label'].value_counts())

In [ ]:
'Deepest Lesion (Recent & Previous) Solar_new'